# MF의 최적 파라미터 찾기

In [6]:
import os
import pandas as pd
import numpy as np 

base_src = '../data/drive-download-20240102T142504Z-001'
u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(
    u_data_src,
    sep='\t',
    names=r_cols,
    encoding='latin-1'
)
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)
# train test split
from sklearn.utils import shuffle
TRAIN_SIZE = 0.75
ratings = shuffle(ratings, random_state=2021)
cutoff = int(TRAIN_SIZE * len(ratings))
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

In [7]:
class NEW_MF():
    def __init__(self, ratings, hyper_params):
        self.R = np.array(ratings)
        # 사용자 수 (num_users)와 아이템 수 (num_items)를 받아온다
        self.num_users, self.num_items = np.shape(self.R)
        # 아래는 MF weight 조절을 위한 하이퍼파라미터
        self.K = hyper_params['K'] # 잠재요인의 수 (latent factor)
        self.alpha = hyper_params['alpha'] # 학습률
        self.beta = hyper_params['beta'] # 정규화 계수
        self.iterations = hyper_params['iterations'] # SGD 반복 횟수
        self.verbose = hyper_params['verbose'] 
        
        item_id_index = []
        index_item_id = []
        for i, one_id in enumerate(ratings):
            item_id_index.append([one_id, i])
            index_item_id.append([i, one_id])
        self.item_id_index = dict(item_id_index)
        self.index_item_id = dict(index_item_id)
        
        user_id_index = []
        index_user_id = []
        for i, one_id in enumerate(ratings.T):
            user_id_index.append([one_id, i])
            index_user_id.append([i, one_id]) 
        self.user_id_index = dict(user_id_index)
        self.index_user_id = dict(index_user_id)
        
    def rmse(self):
        # self.R에서 평점이; 있는(0이 아닌) 요소의 인덱스를 가져옴
        xs, ys = self.R.nonzero()
        # prediction과 error를 담을 리스트 변수 초기화
        self.predictions = []
        self.errors = []
        # 평점이 있는 요소 (사용자 x, 아이템 y) 각각에 대해서 아래의 코드를 실행한다.
        for x, y in zip(xs, ys):
            # 사용자 x 아이템 y에 대해서 평점 예측치를 get_prediction()으로 구한다.
            prediction = self.get_prediction(x, y)
            # 예측치를 prediction에 담고, 실제값과의 오차를 구해서 errors에 담는다.
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction) # 실제값 - 예측값
        # errors를 numpy array로 변환한다.
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        
        return np.sqrt(np.mean(self.errors**2))
   
    def sgd(self):
        for i, j, r in self.samples:
            # 사용자 i, 아이템 j에 대한 평점 예측치 계산
            prediction = self.get_prediction(i, j)
            e = (r - prediction) # (실제값 - 예측값) 오차 계산
            
            # 사용자 평가 경향 계산 및 업데이트
            self.b_u[i] += self.alpha * (e - (self.beta * self.b_u[i]))
            # 아이템 평가 경향 계산 및 업데이트
            self.b_d[j] += self.alpha * (e - (self.beta * self.b_d[j]))
            
            # P 행렬 계산 및 업데이트
            self.P[i, :] += self.alpha * ((e * self.Q[j, :]) - (self.beta * self.P[i, :]))
            # Q 행렬 계산 및 업데이트
            self.Q[j, :] += self.alpha * ((e * self.P[i, :]) - (self.beta * self.Q[j, :]))

    def get_prediction(self, i, j):
        # 사용자 i, 아이템 j에 대한 평점 예측치를 계산한다.
        return self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j,].T)

    # test set
    def set_test(self, ratings_test):
        test_set = []
        for i in range(len(ratings_test)):
            x = self.user_id_index[ratings_test.iloc[i, 0]]
            y = self.item_id_index[ratings_test.iloc[i, 1]]
            z = ratings_test.iloc[i, 2]
            test_set.append([x, y, z])
            # 테스트 셋에으로 추출한 데이터는 원본 행렬에서 평점을 지운다.
            self.R[x, y] = 0
        self.test_set = test_set
        return test_set
    
    # test set에 대한 RMSE 계산
    def test_rmse(self):
        error = 0
        for one_set in self.test_set:
            predicted = self.get_prediction(one_set[0], one_set[1])
            #  pow e => e^2
            error += pow(one_set[2] - predicted, 2)
        return np.sqrt(error/len(self.test_set))
    
    def test(self):
        self.P = np.random.normal(scale= 1./self.K,
                                  size=(self.num_users, self.K))
        self.Q = np.random.normal(scale= 1./self.K,
                                  size=(self.num_items, self.K))
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items) 
        self.b = np.mean(self.R[self.R.nonzero()]) # 전체 평균
        
        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]
        
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse1 = self.rmse()
            rmse2 = self.test_rmse()
            training_process.append((i+1, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print('Iteration: %d ; Train RMSE = %.4f ; Test RMSE = %.4f' % (i+1, rmse1, rmse2))
        return training_process
    
    def get_one_prediction(self, user_id, item_id):
        return self.get_prediction(self.user_id_index[user_id],
                                   self.item_id_index[item_id])

    def full_prediction(self):
        return self.b + self.b_u[:,np.newaxis] + self.b_d[np.newaxis:,] + self.P.dot(self.Q.T)
                                   

In [8]:

# 최적의 K 값 찾기
results = []
index = []

R_temp = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

for K in range(50,261,10):
    print(f'K = {K}')
    hyper_params = {
        'K': K,
        'alpha': 0.001,
        'beta': 0.02,
        'iterations': 300,
        'verbose': True
    }
    mf = NEW_MF(R_temp, hyper_params)
    
    test_set = mf.set_test(ratings_test)
    result = mf.test()
    index.append(K)
    results.append(result)
    

K = 50
Iteration: 10 ; Train RMSE = 0.9669 ; Test RMSE = 0.9807
Iteration: 20 ; Train RMSE = 0.9417 ; Test RMSE = 0.9622
Iteration: 30 ; Train RMSE = 0.9305 ; Test RMSE = 0.9552
Iteration: 40 ; Train RMSE = 0.9239 ; Test RMSE = 0.9515
Iteration: 50 ; Train RMSE = 0.9195 ; Test RMSE = 0.9494
Iteration: 60 ; Train RMSE = 0.9161 ; Test RMSE = 0.9479
Iteration: 70 ; Train RMSE = 0.9131 ; Test RMSE = 0.9469
Iteration: 80 ; Train RMSE = 0.9101 ; Test RMSE = 0.9461
Iteration: 90 ; Train RMSE = 0.9067 ; Test RMSE = 0.9452
Iteration: 100 ; Train RMSE = 0.9024 ; Test RMSE = 0.9441
Iteration: 110 ; Train RMSE = 0.8967 ; Test RMSE = 0.9426
Iteration: 120 ; Train RMSE = 0.8889 ; Test RMSE = 0.9404
Iteration: 130 ; Train RMSE = 0.8785 ; Test RMSE = 0.9375
Iteration: 140 ; Train RMSE = 0.8655 ; Test RMSE = 0.9339
Iteration: 150 ; Train RMSE = 0.8502 ; Test RMSE = 0.9303
Iteration: 160 ; Train RMSE = 0.8332 ; Test RMSE = 0.9269
Iteration: 170 ; Train RMSE = 0.8151 ; Test RMSE = 0.9240
Iteration: 180 ;

KeyboardInterrupt: 

In [ ]:
summary = []
for i in range(len(results)):
    RMSE = []
    for result in results[i]:
        RMSE.append(result[2])
    min = np.min(RMSE)
    j = RMSE.index(min)
    summary.append([index[i], j+1, RMSE[j]])
    